## Applying softmax on the class probabilities 

The title of the notebook pretty much sums the objective. My idea was to extract the class probabilities from the ResNet50 model and applt the softmax function to scale them down and then convert them into discrete predictions. I trained the network for 15 epochs and on the entire training set (no validation set this time). 

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import torch
from fastai.vision import *
from fastai.metrics import *

np.random.seed(7)
torch.cuda.manual_seed_all(7)

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
print(os.listdir("/home/jupyter/fashion_mnist/"))

['train.csv', 'train', 'test.csv', 'submission.csv', 'More experimentation.ipynb', 'A fresh approach.ipynb', 'test', 'models', '.ipynb_checkpoints', 'Identify the apparels.ipynb']


In [4]:
main_path = Path('/home/jupyter/fashion_mnist/')
main_path.ls()

[PosixPath('/home/jupyter/fashion_mnist/train.csv'),
 PosixPath('/home/jupyter/fashion_mnist/train'),
 PosixPath('/home/jupyter/fashion_mnist/test.csv'),
 PosixPath('/home/jupyter/fashion_mnist/submission.csv'),
 PosixPath('/home/jupyter/fashion_mnist/More experimentation.ipynb'),
 PosixPath('/home/jupyter/fashion_mnist/A fresh approach.ipynb'),
 PosixPath('/home/jupyter/fashion_mnist/test'),
 PosixPath('/home/jupyter/fashion_mnist/models'),
 PosixPath('/home/jupyter/fashion_mnist/.ipynb_checkpoints'),
 PosixPath('/home/jupyter/fashion_mnist/Identify the apparels.ipynb')]

In [5]:
train = pd.read_csv(main_path/'train.csv')
sub_file = pd.read_csv(main_path/'test.csv')

In [20]:
test_img = ImageList.from_df(sub_file, path=main_path, folder='test', suffix='.png')

databunch = ImageList.from_df(train, path=main_path, folder='train', suffix='.png')\
            .split_none()\
            .label_from_df()\
            .add_test(test_img)\
            .transform(get_transforms(do_flip=True), size=28)\
            .databunch(path='.', bs=256, device= torch.device('cuda:0'))\
            .normalize()

In [23]:
learner = cnn_learner(databunch, models.resnet50, 
                      metrics=[error_rate, accuracy]).to_fp16()

learner.unfreeze()
learner.fit_one_cycle(15)

In [64]:
preds = learner.get_preds(ds_type=DatasetType.Test)

In [34]:
preds[0].shape

torch.Size([10000, 10])

In [65]:
preds = F.softmax(preds[0], dim=0)

In [66]:
predIdxs = np.argmax(preds.numpy(), axis=1)

In [67]:
np.array(predIdxs)

array([9, 2, 1, 1, ..., 1, 8, 1, 5])

In [68]:
sub_file['label'] = predIdxs
sub_file.to_csv('submission.csv', index=False)

In [69]:
learner.save('best-so-far')

The result was instant:

![](https://i.ibb.co/yqNhm4q/2019-05-07.png)